In [3]:
## Importing Required Libraries

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#Plotly Graphing Libraries
from plotly.offline import init_notebook_mode, iplot
import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')
import plotly.graph_objs as go
import plotly
import plotly.express as px

## Machine Learning Libaries
from sklearn import svm,metrics,tree,preprocessing,linear_model
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split,cross_val_score, cross_val_predict
from sklearn import svm,metrics,tree,preprocessing,linear_model
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import statsmodels.api as sm
from sklearn.metrics import accuracy_score,mean_squared_error,recall_score,confusion_matrix,f1_score,roc_curve, auc
from keras import Sequential
from keras.layers import Dense, Activation
from keras.callbacks import CSVLogger
from sklearn.neural_network import MLPClassifier
from dmba import classificationSummary

# from tensorflow_core.estimator import inputs

no display found. Using non-interactive Agg backend


In [4]:
# Importing the dataset
df = pd.read_csv('DataCoSupplyChainDataset.csv',header= 0,encoding= 'unicode_escape')

In [5]:
df.head()

1


In [6]:
print("s")

s


## Data Description

In [7]:
df.shape

(180519, 53)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180519 entries, 0 to 180518
Data columns (total 53 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Type                           180519 non-null  object 
 1   Days for shipping (real)       180519 non-null  int64  
 2   Days for shipment (scheduled)  180519 non-null  int64  
 3   Benefit per order              180519 non-null  float64
 4   Sales per customer             180519 non-null  float64
 5   Delivery Status                180519 non-null  object 
 6   Late_delivery_risk             180519 non-null  int64  
 7   Category Id                    180519 non-null  int64  
 8   Category Name                  180519 non-null  object 
 9   Customer City                  180519 non-null  object 
 10  Customer Country               180519 non-null  object 
 11  Customer Email                 180519 non-null  object 
 12  Customer Fname                

In [9]:
df.isnull().sum()

Type                                  0
Days for shipping (real)              0
Days for shipment (scheduled)         0
Benefit per order                     0
Sales per customer                    0
Delivery Status                       0
Late_delivery_risk                    0
Category Id                           0
Category Name                         0
Customer City                         0
Customer Country                      0
Customer Email                        0
Customer Fname                        0
Customer Id                           0
Customer Lname                        8
Customer Password                     0
Customer Segment                      0
Customer State                        0
Customer Street                       0
Customer Zipcode                      3
Department Id                         0
Department Name                       0
Latitude                              0
Longitude                             0
Market                                0


###### Combining the Last Name and First names to identify unique customers

In [10]:
df['Cust_Full_Name'] = df['Customer Fname'].astype(str) + df['Customer Lname'].astype(str)

## Data Cleaning

Dropping unimporant columns

In [11]:
df.drop(['Customer Email','Product Status','Customer Password','Customer Street','Customer Fname',
         'Customer Lname','Latitude','Longitude','Product Description','Product Image','Order Zipcode',
         'shipping date (DateOrders)'],axis=1, inplace = True)

In [12]:
df.shape

(180519, 42)

In [13]:
df.isnull().sum()

Type                             0
Days for shipping (real)         0
Days for shipment (scheduled)    0
Benefit per order                0
Sales per customer               0
Delivery Status                  0
Late_delivery_risk               0
Category Id                      0
Category Name                    0
Customer City                    0
Customer Country                 0
Customer Id                      0
Customer Segment                 0
Customer State                   0
Customer Zipcode                 3
Department Id                    0
Department Name                  0
Market                           0
Order City                       0
Order Country                    0
Order Customer Id                0
order date (DateOrders)          0
Order Id                         0
Order Item Cardprod Id           0
Order Item Discount              0
Order Item Discount Rate         0
Order Item Id                    0
Order Item Product Price         0
Order Item Profit Ra

Customer Zipcode has 3 null values which we will fill with 0 as we cannot be sure of the zip code of the customers

In [14]:
df['Customer Zipcode'] = df['Customer Zipcode'].fillna(0)

Creating new column using the Order Date Column

In [15]:
df['order date (DateOrders)'].head()

0    1/31/2018 22:56
1    1/13/2018 12:27
2    1/13/2018 12:06
3    1/13/2018 11:45
4    1/13/2018 11:24
Name: order date (DateOrders), dtype: object

In [16]:
## Splitting Order dates and creating new columns
df['order_yr']= pd.DatetimeIndex(df['order date (DateOrders)']).year
df['order_month'] = pd.DatetimeIndex(df['order date (DateOrders)']).month
df['order_day'] = pd.DatetimeIndex(df['order date (DateOrders)']).weekday
df['order_hour'] = pd.DatetimeIndex(df['order date (DateOrders)']).hour

## Data Visuzalization

In [17]:
plt.style.use("dark_background")
fig, ax = plt.subplots(figsize=(25,10))         # figsize
sns.heatmap(df.corr(),annot=True, linewidths=.3 ,fmt='.1g', cmap= 'coolwarm')

/var/folders/7l/rnr84jk95834djxytnll2h8c0000gn/T/ipykernel_23327/1735471593.py:3: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



<Axes: >

Some products have a negative benefit per order, indicating that the orders are costing the company money.

In [18]:
loss = df[(df['Benefit per order']<0)]

In [19]:
plt.figure(1)
loss['Category Name'].value_counts().nlargest(10).plot.bar(figsize=(15,7), title="Most Loss Making Products")
plt.xticks(rotation = 45)
plt.figure(2)
loss['Market'].value_counts().nlargest(10).plot.bar(figsize=(15,7), title="Most Loss Making Market")
plt.xticks(rotation = 45)

(array([0, 1, 2, 3, 4]),
 [Text(0, 0, 'LATAM'),
  Text(1, 0, 'Europe'),
  Text(2, 0, 'Pacific Asia'),
  Text(3, 0, 'USCA'),
  Text(4, 0, 'Africa')])

Order Status as per the payment types

In [20]:
status = df.groupby('Type')['Order Status'].value_counts()
status_df = status.to_frame()

In [21]:
status_df

Order Status
Type     Order Status                 
CASH     CLOSED                  19616
DEBIT    COMPLETE                59491
         ON_HOLD                  9804
PAYMENT  PENDING_PAYMENT         39832
         PAYMENT_REVIEW           1893
TRANSFER PROCESSING              21902
         PENDING                 20227
         SUSPECTED_FRAUD          4062
         CANCELED                 3692

As we can see that only Transfer payments have a possible suspected fraud situation, what products have the most fraud?

In [22]:
fraud_region = df[(df['Order Status'] == 'SUSPECTED_FRAUD')]

In [23]:
plt.style.use("dark_background")
temp = fraud_region['Category Name'].value_counts().nlargest(10)
temp.iplot(kind='bar', xTitle = 'Category', yTitle = "Count", title = 'Count of Products with suspected fraud', color = '#FF0000')

## Data Modelling and Neural Networks to predict possible fraud

In [24]:
# Creating a copy of the datafram
train_df = df.copy()

In [25]:
# Creating Binary enocode for Suspected Fraud and Late delivery

train_df['fraud'] = np.where(train_df['Order Status'] == 'SUSPECTED_FRAUD', 1, 0)
train_df['late_delivery']=np.where(train_df['Delivery Status'] == 'Late delivery', 1, 0)

In [26]:
train_df.columns

Index(['Type', 'Days for shipping (real)', 'Days for shipment (scheduled)',
       'Benefit per order', 'Sales per customer', 'Delivery Status',
       'Late_delivery_risk', 'Category Id', 'Category Name', 'Customer City',
       'Customer Country', 'Customer Id', 'Customer Segment', 'Customer State',
       'Customer Zipcode', 'Department Id', 'Department Name', 'Market',
       'Order City', 'Order Country', 'Order Customer Id',
       'order date (DateOrders)', 'Order Id', 'Order Item Cardprod Id',
       'Order Item Discount', 'Order Item Discount Rate', 'Order Item Id',
       'Order Item Product Price', 'Order Item Profit Ratio',
       'Order Item Quantity', 'Sales', 'Order Item Total',
       'Order Profit Per Order', 'Order Region', 'Order State', 'Order Status',
       'Product Card Id', 'Product Category Id', 'Product Name',
       'Product Price', 'Shipping Mode', 'Cust_Full_Name', 'order_yr',
       'order_month', 'order_day', 'order_hour', 'fraud', 'late_delivery'],
     

In [27]:
## Removing Identical columns after creating new columns 

train_df.drop(['Delivery Status','Late_delivery_risk','Order Status','order date (DateOrders)'], axis=1, inplace=True)

In [28]:
## Final dimensions of the dataset after wrangling and cleaning

train_df.shape

(180519, 44)

In [29]:
train_df.dtypes

Type                              object
Days for shipping (real)           int64
Days for shipment (scheduled)      int64
Benefit per order                float64
Sales per customer               float64
Category Id                        int64
Category Name                     object
Customer City                     object
Customer Country                  object
Customer Id                        int64
Customer Segment                  object
Customer State                    object
Customer Zipcode                 float64
Department Id                      int64
Department Name                   object
Market                            object
Order City                        object
Order Country                     object
Order Customer Id                  int64
Order Id                           int64
Order Item Cardprod Id             int64
Order Item Discount              float64
Order Item Discount Rate         float64
Order Item Id                      int64
Order Item Produ

### Encoding all Object type variables

In [30]:
le = preprocessing.LabelEncoder()
#convert the categorical columns into numeric
train_df['Customer Country']  = le.fit_transform(train_df['Customer Country'])
train_df['Market']            = le.fit_transform(train_df['Market'])
train_df['Type']              = le.fit_transform(train_df['Type'])
train_df['Product Name']      = le.fit_transform(train_df['Product Name'])
train_df['Customer Segment']  = le.fit_transform(train_df['Customer Segment'])
train_df['Customer State']    = le.fit_transform(train_df['Customer State'])
train_df['Order Region']      = le.fit_transform(train_df['Order Region'])
train_df['Order City']        = le.fit_transform(train_df['Order City'])
train_df['Category Name']     = le.fit_transform(train_df['Category Name'])
train_df['Customer City']     = le.fit_transform(train_df['Customer City'])
train_df['Department Name']   = le.fit_transform(train_df['Department Name'])
train_df['Order State']       = le.fit_transform(train_df['Order State'])
train_df['Shipping Mode']     = le.fit_transform(train_df['Shipping Mode'])
train_df['Order Country']     = le.fit_transform(train_df['Order Country'])
train_df['Cust_Full_Name']    = le.fit_transform(train_df['Cust_Full_Name'])

In [31]:
train_df.head()

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Category Id,Category Name,Customer City,Customer Country,Customer Id,...,Product Name,Product Price,Shipping Mode,Cust_Full_Name,order_yr,order_month,order_day,order_hour,fraud,late_delivery
0,1,3,4,91.250000,314.640015,73,40,66,1,20755,...,78,327.75,3,1876,2018,1,2,22,0,0
1,3,5,4,-249.089996,311.359985,73,40,66,1,19492,...,78,327.75,3,5378,2018,1,5,12,0,1
2,0,4,4,-247.779999,309.720001,73,40,452,0,19491,...,78,327.75,3,4429,2018,1,5,12,0,0
3,1,3,4,22.860001,304.809998,73,40,285,0,19490,...,78,327.75,3,12929,2018,1,5,11,0,0
4,2,2,4,134.210007,298.250000,73,40,66,1,19489,...,78,327.75,3,10638,2018,1,5,11,0,0


## Preparing Data for Neural Networks

Creating a Validation Set from the orignal data

In [32]:
# rows = int(train_df.shape[0] * 0.1)

# randomly select the specified number of rows
# random_rows = np.random.choice(train_df.index, rows, replace=False)

# create a new dataframe from the randomly selected rows
# validation_df = df.loc[random_rows]

# Dropping those rows from the original dataset
# train_df.drop(random_rows,inplace = True)

In [33]:
## Selecting 10% of the dataset for validation

fraction_of_rows = train_df.sample(frac=0.1, random_state = 1)
fraction_of_rows

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Category Id,Category Name,Customer City,Customer Country,Customer Id,...,Product Name,Product Price,Shipping Mode,Cust_Full_Name,order_yr,order_month,order_day,order_hour,fraud,late_delivery
101369,3,2,4,71.849998,197.919998,46,30,529,0,6862,...,67,49.980000,3,9418,2016,4,4,11,0,0
3026,2,5,4,57.529999,164.380005,67,16,66,1,15052,...,17,164.380005,3,7715,2017,11,3,16,0,1
57549,1,5,4,22.680000,226.759995,17,12,66,1,7391,...,71,59.990002,3,8910,2016,10,2,2,0,1
127144,2,6,4,-3.020000,103.989998,18,34,66,1,2588,...,56,129.990005,3,9384,2015,2,5,0,0,1
160375,3,2,4,11.760000,37.560001,75,45,66,1,19769,...,25,39.750000,3,10054,2018,1,2,13,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22603,1,5,4,34.849998,99.580002,17,12,66,1,4792,...,71,59.990002,3,3574,2016,8,5,7,0,1
152893,3,2,1,135.190002,399.980011,45,18,271,0,4055,...,24,399.980011,0,7188,2016,4,0,8,0,1
172935,0,2,2,-39.320000,347.980011,45,18,66,1,148,...,24,399.980011,2,12636,2015,8,5,20,0,0
79244,1,2,1,40.000000,159.990005,48,46,66,1,8580,...,70,199.990005,0,7085,2016,5,4,17,0,1


In [34]:
## Creating a list of index of the samples to be dropped from the main dataset

index = fraction_of_rows.index.values.tolist()
index

[101369,
 3026,
 57549,
 127144,
 160375,
 21822,
 87233,
 12914,
 51333,
 18605,
 115091,
 125669,
 22070,
 11734,
 58228,
 138444,
 85450,
 73875,
 61313,
 47730,
 149530,
 132499,
 48403,
 83521,
 156055,
 90164,
 82899,
 72305,
 17851,
 108983,
 100037,
 102209,
 29548,
 19874,
 154032,
 51027,
 155354,
 16041,
 170095,
 6833,
 10017,
 79882,
 5546,
 77913,
 178418,
 151798,
 130796,
 82481,
 105142,
 128126,
 137384,
 75556,
 138450,
 160802,
 130777,
 73925,
 42290,
 63988,
 89053,
 93300,
 172089,
 18939,
 59310,
 10681,
 59937,
 2498,
 174639,
 128056,
 26597,
 175972,
 17466,
 107272,
 94543,
 42112,
 160282,
 165595,
 50653,
 12994,
 60858,
 15458,
 139747,
 40511,
 84754,
 81681,
 69153,
 42067,
 84171,
 64924,
 115560,
 173574,
 133262,
 26419,
 165598,
 173499,
 79388,
 123411,
 47458,
 128503,
 177848,
 129002,
 115929,
 177510,
 72672,
 89007,
 136163,
 158069,
 15232,
 161432,
 32315,
 49397,
 38558,
 80601,
 20518,
 162135,
 26052,
 131531,
 91792,
 158439,
 84594,
 20

In [35]:
## Dropping rows with the index numbers

train_df.drop(index = index, inplace = True)

In [36]:
train_df.shape

(162467, 44)

In [37]:
## Resetting index for the validation dataset

fraction_of_rows.reset_index(drop = True, inplace=True)

In [38]:
fraction_of_rows.shape

(18052, 44)

In [39]:
## Creating X and y for dependant and independent variables

X = train_df.loc[:,train_df.columns != 'fraud']
y = train_df['fraud']

In [40]:
## Splitting the dataset into training and test data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state=42)

In [41]:
## Creating dataframe for X and y validation dataset

X_valid = fraction_of_rows.loc[:,fraction_of_rows.columns != 'fraud']
y_valid = fraction_of_rows['fraud']

In [42]:
X_train.shape

(113726, 43)

In [43]:
# Defining the classes 
classes = sorted(y_train.unique())

### Standardizing the data

In [44]:
## Standardizing the X dataset

ss = StandardScaler()
X_train=ss.fit_transform(X_train)
X_test=ss.transform(X_test)
X_valid=ss.transform(X_valid)

## Creating MLPClassifier Model

In [45]:
clf = MLPClassifier(hidden_layer_sizes=(6), activation='logistic', solver='lbfgs',random_state=1)

In [46]:




## Fitting the data using MLPClassifier

clf.fit(X_train, y_train)

MLPClassifier(activation='logistic', hidden_layer_sizes=6, random_state=1,
              solver='lbfgs')

In [47]:
## Predicting the trained model

clf.predict(X_test)

array([0, 0, 0, ..., 0, 0, 0])

In [48]:
## Creating a Classification matrix for MLPClassifier

classificationSummary(y_test, clf.predict(X_test), class_names=classes)

Confusion Matrix (Accuracy 0.9789)

       Prediction
Actual     0     1
     0 47280   330
     1   697   434


## Creating a Custom Neural Network Model

In [49]:
train_df.shape

(162467, 44)

In [50]:
keras.layers.BatchNormalization()
model = Sequential()
#First Hidden Layer
model.add(Dense(1024, activation='relu', kernel_initializer='random_normal', input_dim=43)) #As we have 43 columns

#All other hidden layers in a for loop with nodes reducing in each loop
nodes = 1024
for i in range(9):
    nodes = nodes // 2
    model.add(Dense(nodes, activation='relu', kernel_initializer='random_normal'))

#Output Layer
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

/Users/vishal.r001/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 1024)           │        45,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 4)              │            36 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 2)              │            10 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 1)              │             3 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 1)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 745,129 (2.84 MB)

 Trainable params: 745,129 (2.84 MB)

 Non-trainable params: 0 (0.00 B)

As F1 score is not accessible in Keras, binary crossentropy is used to measure loss and accuracy because the output data is binary classification.

In [51]:
## Compiling the model created

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

### Fitting the model using custom neural network

Intitally we'll tried with 10 epochs which had a good accuracy but the loss was high, then we increased it to 15 which reduced the loss significantly and we stopped there to avoid overfitting of the data. 

In [52]:
csvlogger = CSVLogger('training.log',separator=',',append=False)

result = model.fit(X_train, y_train, batch_size = 512, epochs = 15, callbacks=[csvlogger])

Epoch 1/15
223/223 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.9765 - loss: 0.3368
Epoch 2/15
223/223 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.9776 - loss: 0.0554
Epoch 3/15
223/223 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.9773 - loss: 0.0528
Epoch 4/15
223/223 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.9778 - loss: 0.0491
Epoch 5/15
223/223 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.9775 - loss: 0.0488
Epoch 6/15
223/223 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.9777 - loss: 0.0469
Epoch 7/15
223/223 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.9780 - loss: 0.0447
Epoch 8/15
223/223 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.9776 - loss: 0.0429
Epoch 9/15
223/223 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.9784 - loss: 0.0417
Epoch 10/15
223/223 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - accuracy: 0.9822 - loss: 0.0391
Epoch 11/15
223/223 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.9832 - loss: 0.0357
Epoch 12/15
223/223 ━━━━━━━━━━━━━━━━━━━━ 

In [53]:
plt.plot(result.history['accuracy'],'green',label='Accuracy')
plt.plot(result.history['loss'],'red',label='Loss')
plt.title('Training Accuracy & Loss')
plt.xlabel('Epoch')
plt.legend(loc=0)

In [54]:
# Predicting the custom model model

train_evaluate=model.evaluate(X_train, y_train)
test_evaluate=model.evaluate(X_test, y_test)
print('accuracy for Train set is',train_evaluate)
print('accuracy for Test set is',test_evaluate) # evaluation of model.
yf_pred1=model.predict(X_test,batch_size=512,verbose=1)
yf_pred=np.argmax(yf_pred1,axis=1)
print(f1_score(y_test,yf_pred,average="weighted"))

3554/3554 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.9882 - loss: 0.0261
1524/1524 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9790 - loss: 0.0674
accuracy for Train set is [0.026415517553687096, 0.9878831505775452]
accuracy for Test set is [0.06531106680631638, 0.9784780740737915]
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
0.9653297639890498


As we can see that the Train and test accuracy is very high at ~99% and loss is at ~8.88%.  
Along with that the F1 Score calculated is 96.58%. 

## Validating the Models

### MLP Classifier

In [55]:
## Predicting the validation dataset

clf_pred = clf.predict(X_valid)
clf_pred

array([0, 0, 0, ..., 0, 0, 0])

In [56]:
## Classification Matrix for Validation Dataset

classificationSummary(y_valid, clf.predict(X_valid), class_names=classes)

Confusion Matrix (Accuracy 0.9787)

       Prediction
Actual     0     1
     0 17532   130
     1   254   136


In [57]:
accuracy_score(y_valid,clf_pred)

0.9787281187680036

### Custom NN

In [58]:
predictions = model.predict(X_valid)

565/565 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step


In [59]:
predictions = (predictions > 0.5).astype(np.float32)

In [60]:
predictions

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

In [61]:
accuracy_score(y_valid,predictions)

0.9795036561045868

In [62]:
keras.backend.clear_session()